In [1]:
import os
%pwd

'd:\\End_to_End\\textsummarisation\\research'

In [2]:
os.chdir("../")

In [3]:
%pwd

'd:\\End_to_End\\textsummarisation'

In [22]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    tokenizer_name: Path

In [23]:
#Configuratoin manager

from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml,create_directories


In [24]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_PATH):

        self.config= read_yaml(config_filepath)
        self.params= read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
    
    def get_data_transformation_config(self):
        config=self.config.data_transformation

        create_directories([config.root_dir])

        data_tranformation_config=DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            tokenizer_name=config.tokenizer_name
        )

        return data_tranformation_config

In [25]:
import os
from textSummarizer.logging import logger
from transformers import AutoTokenizer
from datasets import load_dataset,load_from_disk

In [44]:
class DataTransformation:
    def __init__(self, config:DataTransformationConfig):
        self.config = config
        self.tokenizer = AutoTokenizer.from_pretrained(config.tokenizer_name)

    def convert_examples_to_features(self,example_batch):
        input_encodings=self.tokenizer(example_batch['dialogue'], max_length=512, truncation=True,padding='max_length',return_tensors='pt')

        with self.tokenizer.as_target_tokenizer():
            target_encodings=self.tokenizer(example_batch['summary'], max_length=64, truncation=True, padding='max_length', return_tensors='pt')
        return{
            'input_ids' : input_encodings['input_ids'],
            'attention_maks' : input_encodings['attention_mask'],
            'labels' : target_encodings['input_ids']
        }
    def convert(self):
        dataset_samsum=load_from_disk(self.config.data_path)
        dataset_samsum_pt=dataset_samsum.map(self.convert_examples_to_features, batched=True)
        dataset_samsum_pt.save_to_disk(os.path.join(self.config.root_dir,"samsum_dataset"))

In [45]:
#Pipeline

try:
    config=ConfigurationManager()
    data_transformation_config=config.get_data_transformation_config()
    data_transformation=DataTransformation(config=data_transformation_config)
    data_transformation.convert()
except Exception as e:
    raise e

[2025-11-14 00:02:59,757: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-11-14 00:02:59,761: INFO: common: yaml file: params.yaml loaded successfully]
[2025-11-14 00:02:59,763: INFO: common: created directory at: artifacts]
[2025-11-14 00:02:59,765: INFO: common: created directory at: artifacts/data_transformation]


d:\End_to_End\textsummarisation\venv\Lib\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Map:   0%|          | 0/14732 [00:00<?, ? examples/s]d:\End_to_End\textsummarisation\venv\Lib\site-packages\transformers\tokenization_utils_base.py:4117: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
Saving the dataset (1/1 shards): 100%|██████████| 818/818 [00:00<00:00, 44210.87 examples/s]
